This notebook describes a classifier for whether the Turkish “de/da” and “ki” suffixes should be
separated or not.

For example, in “Öğrenciler de geldi” it is separated, but in “Öğrencilerde gelişme
var.” it is not separated.

# Imports
*   Import necessary libraries



In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Providing the Data

In [12]:
# Read the annotated file
with open("annotated_dataset_for_de_ki.txt", "r", encoding="utf-8") as file:
    lines = file.readlines()

# Data pre-processing
sentences = []
labels = []

for line in lines:
    parts = line.strip().split()
    if len(parts) >= 2:
        sentences.append(parts[0])
        labels.append(parts[-1])

# Convert labels to numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(labels)

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1

# Convert sentences to sequences
X = tokenizer.texts_to_sequences(sentences)
X = pad_sequences(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and Compile the Neural Network

In [14]:
# Build the model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=X.shape[1]))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))  # Output layer with softmax for multi-class

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Training the Neural Network

In [15]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/10
118/118 [==============================] - 7s 28ms/step - loss: 0.5013 - accuracy: 0.9463 - val_loss: 0.1852 - val_accuracy: 0.9593
Epoch 2/10
118/118 [==============================] - 1s 11ms/step - loss: 0.1727 - accuracy: 0.9577 - val_loss: 0.1411 - val_accuracy: 0.9641
Epoch 3/10
118/118 [==============================] - 1s 11ms/step - loss: 0.1124 - accuracy: 0.9606 - val_loss: 0.1427 - val_accuracy: 0.9665
Epoch 4/10
118/118 [==============================] - 1s 11ms/step - loss: 0.0937 - accuracy: 0.9636 - val_loss: 0.1298 - val_accuracy: 0.9689
Epoch 5/10
118/118 [==============================] - 1s 11ms/step - loss: 0.0894 - accuracy: 0.9668 - val_loss: 0.1266 - val_accuracy: 0.9713
Epoch 6/10
118/118 [==============================] - 1s 10ms/step - loss: 0.0834 - accuracy: 0.9665 - val_loss: 0.1293 - val_accuracy: 0.9713
Epoch 7/10
118/118 [==============================] - 1s 10ms/step - loss: 0.0762 - accuracy: 0.9689 - val_loss: 0.1309 - val_accuracy: 0.9713

# Save the Model

In [16]:
import pickle

# Specify the path where you want to save the model
model_save_path = "/content/custom_model/model4.h5"

# Save the model
model.save(model_save_path)

# Optionally, you can also save the tokenizer if you need it for later use
tokenizer_save_path = "/content/custom_model/tokenizer4.pickle"
with open(tokenizer_save_path, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Test the Model

In [18]:
from keras.models import load_model

# Load the model
loaded_model4 = load_model(model_save_path)

# Optionally, load the tokenizer
with open(tokenizer_save_path, 'rb') as handle:
    loaded_tokenizer = pickle.load(handle)


# Load and predict on sentences from "test_sentences_de.txt"
test_sentences_path = "test_sentences_de_ki.txt"

with open(test_sentences_path, 'r', encoding='utf-8') as file:
    test_sentences = file.readlines()

treshold = 3.0732085072784685e-06

# Predictions
for sentence_to_predict in test_sentences:
  print(f'\nInput Sentence: {sentence_to_predict}')
  X_new = tokenizer.texts_to_sequences([sentence_to_predict])
  X_new = pad_sequences(X_new, maxlen=X.shape[1])

  prediction = model.predict(X_new)[0][0]
  prediction_label = 'Error Detected! Separate it!' if prediction < treshold else 'Correct :)'

  print(f'Prediction: {prediction_label} (Probability: {prediction})')


Input Sentence: Evde yağ kalmamış.

1/1 [==============================] - 0s 28ms/step
Prediction: Correct :) (Probability: 5.9723766753450036e-05)

Input Sentence: Bende gelmek istiyordum.

1/1 [==============================] - 0s 22ms/step
Prediction: Correct :) (Probability: 5.9723766753450036e-05)

Input Sentence: İnsanın bazen çıkıp gezmeside gerekiyor.

1/1 [==============================] - 0s 21ms/step
Prediction: Error Detected! Separate it! (Probability: 2.7732085072784685e-06)

Input Sentence: Anahtarları piknikte kaybetmiş.

1/1 [==============================] - 0s 21ms/step
Prediction: Correct :) (Probability: 5.9723766753450036e-05)

Input Sentence: Babam geldiğinde gidelim.

1/1 [==============================] - 0s 20ms/step
Prediction: Correct :) (Probability: 5.9723766753450036e-05)

Input Sentence: Sadece hastalandığımda arıyorsun.

1/1 [==============================] - 0s 20ms/step
Prediction: Correct :) (Probability: 2.648703593877144e-05)

Input Sentence: Bab